In [10]:
import os
from skimage import io
import numpy as np

def extract_patches(input_dir, output_dir, patch_size=(128, 128), stride=None):
    
    os.makedirs(output_dir, exist_ok=True)
    stride = stride or patch_size
    count = 0
    for fname in os.listdir(input_dir):
        path = os.path.join(input_dir, fname)
        img = io.imread(path)
        h, w = img.shape[:2]
        for y in range(0, h - patch_size[0] + 1, stride[0]):
            for x in range(0, w - patch_size[1] + 1, stride[1]):
                patch = img[y:y+patch_size[0], x:x+patch_size[1]]
                out_path = os.path.join(output_dir, f"probka_{count:04d}.png")
                io.imsave(out_path, patch)
                count += 1

for category in ["blaty", "biurka", "plytki"]:
    extract_patches(
        input_dir=f"tekstury/{category}",
        output_dir=f"probki/{category}",
        patch_size=(128, 128),
        stride=(128, 128)
    )

/var/folders/dm/pfnq90993sqfvqbxbplj919h0000gn/T/ipykernel_26940/3711794263.py:18: UserWarning: probki/plytki/probka_0000.png is a low contrast image
  io.imsave(out_path, patch)
/var/folders/dm/pfnq90993sqfvqbxbplj919h0000gn/T/ipykernel_26940/3711794263.py:18: UserWarning: probki/plytki/probka_0001.png is a low contrast image
  io.imsave(out_path, patch)
/var/folders/dm/pfnq90993sqfvqbxbplj919h0000gn/T/ipykernel_26940/3711794263.py:18: UserWarning: probki/plytki/probka_0007.png is a low contrast image
  io.imsave(out_path, patch)
/var/folders/dm/pfnq90993sqfvqbxbplj919h0000gn/T/ipykernel_26940/3711794263.py:18: UserWarning: probki/plytki/probka_0012.png is a low contrast image
  io.imsave(out_path, patch)
/var/folders/dm/pfnq90993sqfvqbxbplj919h0000gn/T/ipykernel_26940/3711794263.py:18: UserWarning: probki/plytki/probka_0013.png is a low contrast image
  io.imsave(out_path, patch)
/var/folders/dm/pfnq90993sqfvqbxbplj919h0000gn/T/ipykernel_26940/3711794263.py:18: UserWarning: probki/pl

In [12]:
import os
import numpy as np
import pandas as pd
from skimage import io, color, util
from skimage.feature import graycomatrix, graycoprops

VALID_EXTS = ('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')

def compute_glcm_features(img_gray, distances, angles, levels=64):
    
    
    if img_gray.dtype != np.uint8:
        img = util.img_as_ubyte(img_gray)
    else:
        img = img_gray
    img = (img / 255.0 * (levels - 1)).astype(np.uint8)
    
    glcm = graycomatrix(img, 
                        distances=distances, 
                        angles=angles, 
                        levels=levels, 
                        symmetric=True, 
                        normed=True)
    feats = {}
    props = ['dissimilarity', 'correlation', 'contrast',
             'energy', 'homogeneity', 'ASM']
    for prop in props:
        vals = graycoprops(glcm, prop)
        feats[prop] = vals.mean()  
    return feats


rows = []
categories = os.listdir("probki")
distances = [1, 3, 5]
angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
print(categories)
categories.pop(0)
print(categories)
for cat in categories:
    folder = os.path.join("probki", cat)
    for fname in os.listdir(folder):
        if fname.startswith('.') or not fname.lower().endswith(VALID_EXTS):
            continue
        img = io.imread(os.path.join(folder, fname))
        gray = color.rgb2gray(img)
        feats = compute_glcm_features(gray, distances, angles, levels=64)
        feats['category'] = cat
        rows.append(feats)

df = pd.DataFrame(rows)
df.to_csv("cechy_tekstur.csv", index=False)
print("Zapisano cechy do cechy_tekstur.csv")


['.DS_Store', 'blaty', 'biurka', 'plytki']
['blaty', 'biurka', 'plytki']
Zapisano cechy do cechy_tekstur.csv


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report


df = pd.read_csv("cechy_tekstur.csv")
X = df.drop(columns=['category']).values
y = LabelEncoder().fit_transform(df['category'])


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y)

scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)

clf = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
clf.fit(X_train_s, y_train)

y_pred = clf.predict(X_test_s)
acc = accuracy_score(y_test, y_pred)
print(f"Dokładność klasyfikatora: {acc:.2%}\n")
print("Szczegółowy raport:\n", classification_report(y_test, y_pred))

Dokładność klasyfikatora: 97.56%

Szczegółowy raport:
               precision    recall  f1-score   support

           0       0.94      0.99      0.97       642
           1       0.99      0.94      0.96       642
           2       1.00      1.00      1.00       642

    accuracy                           0.98      1926
   macro avg       0.98      0.98      0.98      1926
weighted avg       0.98      0.98      0.98      1926

